In [1]:
import torch
import functions
import modules
import math
from optimizer import SGD

from torch import optim
from torch import Tensor
from torch import nn

# torch.set_grad_enabled(False)

- Generates a training and a test set of 1,000 points sampled uniformly in [0, 1]2, each with a
label 0 if outside the disk of radius 1/√2π and 1 inside,
- two input units, two output units, three hidden layers of 25 units,

## Test.py

## Generate data

In [2]:
def generate_disc_set(nb):
    input = torch.Tensor(nb, 2).uniform_(0, 1)
    target = input.pow(2).sum(1).sub(1 / (2 * math.pi)).sign().add(1).div(2).long()
    return input, target

In [3]:
def convert_to_one_hot_labels(input, target):
    tmp = input.new_zeros(target.size(0), target.max() + 1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

In [4]:
train_input, train_target = generate_disc_set(3)

In [5]:
print(train_input)
print(train_target)

tensor([[0.5978, 0.0330],
        [0.4198, 0.6532],
        [0.1504, 0.1193]])
tensor([1, 1, 0])


In [6]:
train_target = convert_to_one_hot_labels(train_input, train_target)

In [7]:
print(train_input)
print(train_target)

tensor([[0.5978, 0.0330],
        [0.4198, 0.6532],
        [0.1504, 0.1193]])
tensor([[0., 1.],
        [0., 1.],
        [1., 0.]])


In [8]:
train_input, train_target = generate_disc_set(1000)
train_target = convert_to_one_hot_labels(train_input, train_target)

In [9]:
mean, std = train_input.mean(), train_input.std()
train_input.sub_(mean).div_(std)

tensor([[ 1.6113,  0.7463],
        [ 0.4971,  0.8789],
        [ 0.0578, -1.5177],
        ...,
        [ 1.4818,  0.3531],
        [ 0.3349,  0.4327],
        [-0.6696,  0.6123]])

In [10]:
train_input = 0.9 * train_input

In [11]:
print(train_input.shape)
print(train_target.shape)

torch.Size([1000, 2])
torch.Size([1000, 2])


In [12]:
arch1 = modules.Sequential(modules.Linear(2, 25), modules.Linear(25, 2), modules.MSELoss(train_target))

In [13]:
for name, m in arch1.__dict__.items():
    print(name)
    print(m)

Linear0
Linear1
MSELoss2


In [14]:
nb_epochs = 2
lr = 1e-6
# optimizer = test(arch1.param(), lr=lr)

In [15]:
class TEST(object):
    def __init__(self, params):
        self.params = params
        super().__init__()
        
    def modify(self):
        lr = 1e-6
        params_tmp = []
        for (p, grad2) in self.params:
            p -= lr * grad2
            grad2 = torch.zeros(grad2.shape)
            params_tmp.append((p, grad2))
        self.params = params_tmp

In [16]:
optim = TEST(arch1.param())
for e in range(nb_epochs):
    output, loss = arch1.forward(train_input)
    grad = arch1.backward()
    optim.modify()

In [17]:
# nb_epochs = 2
# lr = 1e-3
# optimizer = SGD(arch1.param(), lr=lr)

# for e in range(nb_epochs):
#     output, loss = arch1.forward(train_input)
#     print(loss)
#     grad = arch1.backward()
#     for (p, grad2) in arch1.param():
#         p -= lr * grad2
#         grad2 = torch.zeros(grad2.shape)

In [18]:
nb_epochs = 50
lr = 1e-5
optimizer = SGD(arch1.param(), lr=lr)

In [19]:
for e in range(nb_epochs):
    output, loss = arch1.forward(train_input)
    print(loss)
    grad = arch1.backward()
    optimizer.step()
    optimizer.zero_grad()

tensor(1.7430)
tensor(1.5540)
tensor(1.3781)
tensor(1.2148)
tensor(1.0641)
tensor(0.9257)
tensor(0.7995)
tensor(0.6851)
tensor(0.5825)
tensor(0.4913)
tensor(0.4115)
tensor(0.3428)
tensor(0.2850)
tensor(0.2379)
tensor(0.2013)
tensor(0.1750)
tensor(0.1589)
tensor(0.1527)
tensor(0.1563)
tensor(0.1695)
tensor(0.1921)
tensor(0.2238)
tensor(0.2647)
tensor(0.3143)
tensor(0.3727)
tensor(0.4396)
tensor(0.5148)
tensor(0.5982)
tensor(0.6895)
tensor(0.7888)
tensor(0.8956)
tensor(1.0100)
tensor(1.1317)
tensor(1.2606)
tensor(1.3965)
tensor(1.5393)
tensor(1.6888)
tensor(1.8449)
tensor(2.0073)
tensor(2.1761)
tensor(2.3509)
tensor(2.5317)
tensor(2.7183)
tensor(2.9106)
tensor(3.1085)
tensor(3.3117)
tensor(3.5202)
tensor(3.7339)
tensor(3.9525)
tensor(4.1760)


In [20]:
model = nn.Sequential(
            nn.Linear(2, 2),
#             nn.ReLU(),
#             nn.Linear(128, 2)
        )
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)
nb_epochs = 50

for e in range(nb_epochs):
    output = model(train_input)
#     print(output)
    loss = criterion(output, train_target.float())
    print(loss)
    model.zero_grad()
    loss.backward()
    optimizer.step()

AttributeError: 'TEST' object has no attribute 'SGD'

## Mini-Project Architecture

In [ ]:
architecture = modules.Sequential(modules.Linear(2, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 2),
                                  modules.MSELoss()
                                 )